In [1]:
from planning_inference.parsers import parse_model, parse_problem, parse_plan, parse_trajectory, parse_hypothesis, parse_observation_sequence
from planning_inference.generator import generate_trajectory
from planning_inference.functions import generate_all_literals, get_matching_literals

from planning_inference.pddl import Conjunction, Literal, Type, TypedObject, Effect, Truth, NumericConstant, PrimitiveNumericExpression, Increase
from planning_inference.pddl import SensorModel

from planning_inference.observations import Trajectory, Hypothesis, State

from planning_inference import DecodingTask

from sensor_models import load_sensor_model

import os
import copy
from collections import defaultdict
from itertools import combinations
from random import choice, choices, shuffle
from statistics import mean
import glob
import time

In [2]:
def launch_experiments(domain, task, timeout):
    Ms = load_sensor_model(domain)
    
    observabilities = [30, 50, 70]
       
    for observability in observabilities:
        base_path = "benchmark/%s/%s/%s/" % (domain, task, str(observability))

        problems = sorted(glob.glob(base_path + "*"))

        for problem in problems:
            costs = []
            times = []

            print(problem)

            # Planning Model
            Mp = parse_model(problem + "/domain")
            
            # Real Hypothesis
            with open(problem + "/sol", "r") as f:
                correct_h = int(f.read())

            print("Correct hypothesis: %d" % correct_h)

            # Observation sequence
            obs = parse_observation_sequence(problem + "/obs")

            # Hypotheses
            h_files = sorted(glob.glob(problem + "/hyp*"))

            for i in range(len(h_files)):
                h = parse_hypothesis(h_files[i])

                #Build a decoding problem for each hypothesis
                T = DecodingTask(Mp,Ms,obs,h)

                tic = time.time()
                sol = T.decode(clean=True, planner="downward", t=timeout)
                toc = time.time()


                if len(sol.actions) == 0:
                    cost = 1000
                else:
                    cost = len(sol.actions)
                    sol.to_file(problem + "/plan.%s" % str(i).zfill(2))

                duration = toc - tic

                print("Hypothesis %d: %d, %.2f" % (i, cost, duration))

                costs.append(cost)
                times.append(duration)

            with open(problem + "/costs", "w") as f:
                f.write(" ".join(map(str, costs)))

            with open(problem + "/times", "w") as f:
                f.write(" ".join(map(str, times)))
    

In [3]:
domain = "grid"
timeout = 120

In [4]:
# MONITORING

task = "monitoring"
launch_experiments(domain, task, timeout)

benchmark/grid/monitoring/30/P00
Correct hypothesis: 0
Hypothesis 0: 19, 2.57
Hypothesis 1: 1000, 0.96
Hypothesis 2: 1000, 1.21
Hypothesis 3: 1000, 0.98
Hypothesis 4: 1000, 1.07
benchmark/grid/monitoring/30/P01
Correct hypothesis: 0
Hypothesis 0: 13, 2.49
Hypothesis 1: 1000, 1.08
Hypothesis 2: 1000, 0.82
Hypothesis 3: 1000, 1.35
Hypothesis 4: 25, 2.40
benchmark/grid/monitoring/30/P02
Correct hypothesis: 5
Hypothesis 0: 1000, 1.44
Hypothesis 1: 1000, 1.08
Hypothesis 2: 1000, 0.96
Hypothesis 3: 1000, 1.08
Hypothesis 4: 1000, 0.97
Hypothesis 5: 16, 2.18
benchmark/grid/monitoring/30/P03
Correct hypothesis: 0
Hypothesis 0: 20, 6.03
Hypothesis 1: 1000, 1.23
Hypothesis 2: 1000, 2.19
Hypothesis 3: 1000, 1.27
Hypothesis 4: 1000, 1.23
Hypothesis 5: 1000, 1.99
benchmark/grid/monitoring/30/P04
Correct hypothesis: 2
Hypothesis 0: 1000, 1.05
Hypothesis 1: 1000, 0.82
Hypothesis 2: 15, 1.86
Hypothesis 3: 1000, 0.84
Hypothesis 4: 20, 1.87
benchmark/grid/monitoring/30/P05
Correct hypothesis: 2
Hypothesi

In [5]:
# PREDICTION

task = "prediction"
launch_experiments(domain, task, timeout)

benchmark/grid/prediction/30/P00
Correct hypothesis: 0
Hypothesis 0: 33, 2.76
Hypothesis 1: 37, 3.71
benchmark/grid/prediction/30/P01
Correct hypothesis: 1
Hypothesis 0: 30, 2.67
Hypothesis 1: 26, 2.30
benchmark/grid/prediction/30/P02
Correct hypothesis: 1
Hypothesis 0: 33, 3.09
Hypothesis 1: 29, 3.87
benchmark/grid/prediction/30/P03
Correct hypothesis: 0
Hypothesis 0: 27, 7.75
Hypothesis 1: 35, 7.78
benchmark/grid/prediction/30/P04
Correct hypothesis: 1
Hypothesis 0: 31, 4.07
Hypothesis 1: 27, 2.76
benchmark/grid/prediction/30/P05
Correct hypothesis: 1
Hypothesis 0: 30, 113.81
Hypothesis 1: 26, 85.16
benchmark/grid/prediction/30/P06
Correct hypothesis: 0
Hypothesis 0: 33, 4.21
Hypothesis 1: 37, 3.07
benchmark/grid/prediction/30/P07
Correct hypothesis: 0
Hypothesis 0: 31, 4.43
Hypothesis 1: 35, 2.86
benchmark/grid/prediction/30/P08
Correct hypothesis: 0
Hypothesis 0: 35, 3.86
Hypothesis 1: 39, 3.15
benchmark/grid/prediction/30/P09
Correct hypothesis: 0
Hypothesis 0: 32, 2.82
Hypothesis

In [6]:
# HINDSIGHT

task = "hindsight"
launch_experiments(domain, task, timeout)

benchmark/grid/hindsight/30/P00
Correct hypothesis: 1
Hypothesis 0: 1000, 123.36
Hypothesis 1: 34, 3.97
benchmark/grid/hindsight/30/P01
Correct hypothesis: 0
Hypothesis 0: 34, 8.05
Hypothesis 1: 1000, 122.25
benchmark/grid/hindsight/30/P02
Correct hypothesis: 0
Hypothesis 0: 32, 3.85
Hypothesis 1: 1000, 121.79
benchmark/grid/hindsight/30/P03
Correct hypothesis: 1
Hypothesis 0: 1000, 121.77
Hypothesis 1: 30, 5.20
benchmark/grid/hindsight/30/P04
Correct hypothesis: 0
Hypothesis 0: 30, 6.31
Hypothesis 1: 1000, 121.56
benchmark/grid/hindsight/30/P05
Correct hypothesis: 1
Hypothesis 0: 1000, 122.26
Hypothesis 1: 31, 3.67
benchmark/grid/hindsight/30/P06
Correct hypothesis: 0
Hypothesis 0: 36, 3.79
Hypothesis 1: 1000, 122.16
benchmark/grid/hindsight/30/P07
Correct hypothesis: 0
Hypothesis 0: 38, 88.22
Hypothesis 1: 1000, 121.78
benchmark/grid/hindsight/30/P08
Correct hypothesis: 1
Hypothesis 0: 1000, 124.59
Hypothesis 1: 40, 4.89
benchmark/grid/hindsight/30/P09
Correct hypothesis: 0
Hypothesi